In [1]:
#Import Libraries for notebook use
import pandas as pd
import psycopg2
import numpy as np
import csv

In [2]:
#Type out queries to get an understanding of the tables
query_artist =  """
        SELECT * FROM artist 
        LIMIT 10
        ;
"""

query_label =  """
        SELECT * FROM label
        ;
"""

query_release =  """
        SELECT * FROM release
        LIMIT 10
        ;
"""

query_releases_artists =  """
        SELECT * FROM releases_artists
        LIMIT 10
        ;
"""

query_releases_labels =  """
        SELECT * FROM releases_labels
        LIMIT 10
        ;
"""


Find the set of major labels in the Discogs database

In [3]:
#Connect to the db and execute query
try:
#      removed connection information bc it 
#      contained username/password + reran to remove information from print out
    connection = psycopg2.connect() 

    cursor = connection.cursor()
    # Print PostgreSQL Connection properties
    print ( connection.get_dsn_parameters(),"\n")
    
    # Print PostgreSQL version
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record,"\n")
    
    #EXECUTE YOUR QUERY AND SAVE RESULTS IN DATAFRAME
    df_label = pd.read_sql_query(query_label,con=connection)
    
except (Exception, psycopg2.Error) as error :
    print("Error while connecting to PostgreSQL", error)
finally:
    #closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

Error while connecting to PostgreSQL missing dsn and no parameters


NameError: name 'connection' is not defined

The "majors" list is based upon info I found on wikipedia on the names of the sub-labels within each of the big 3 music groups:
https://en.wikipedia.org/wiki/List_of_Universal_Music_Group_labels
https://en.wikipedia.org/wiki/List_of_Sony_Music_labels
https://en.wikipedia.org/wiki/Warner_Music_Group

In [4]:
#Gather the set of all major labels stored in the Discogs database
majors = ['SONY','UNIVERSAL','WARNER', 'COLUMBIA', 'AMERICAN RECORD CORPORATION', 'CBS', 'EPIC RECORDS',
         'RCA RECORDS', 'INTERSCOPE', 'GEFFEN', 'CAPITOL', 'REPUBLIC RECORDS',
         'ISLAND RECORDS', 'DEF JAM', 'CAROLINE RECORDS', 'ISOLATION NETWORK',
         'BMG', 'ELEKTRA', 'PARLOPHONE', 'ATLANTIC RECORDS']

#Iterate through all the names and parent_labels. If any of them point to the majors, store it in sets
majors_found = 0
major_set = set()
major_label_ids = set()

for _id, name, parent in zip(list(df_label.id), list(df_label.name), list(df_label.parent_label)):
    found = False
    for major in majors:
        if major in name.upper():
            #print(_id, ' major label found: ', name)
            major_set.add(name)
            major_label_ids.add(_id)
            found = True
        if parent is not None: 
            if major in parent.upper():
                #print(_id, ' major parent found: ', parent)
                major_set.add(parent)
                major_label_ids.add(_id)
                found = True
    if found == True:
        majors_found += 1

    

Now, we want to ISOLATE the artists whose FIRST major release was a country release in or after 2017.
Note the following:
They should not have a release with a major label prior to 2017

In [5]:
#Build and execute the queries for this task
#Build the query string of major label ids
#from psycopg2 import sql

major_label_ids_search_string = str(list(major_label_ids))

major_label_ids_search_string = major_label_ids_search_string[1:-1]

find_major_country_releases_since_2017 = ("SELECT a.id, a.name, r.title, r.genres,\
                       r.status, r.styles, r.released, r.country,\
                       ra.position, l.name AS ""label_name"", l.parent_label, \
                       SUBSTR(r.released, 1, 4) AS ""Year"" \
                  FROM artist a \
            INNER JOIN releases_artists ra ON a.id = ra.artist_id \
            INNER JOIN releases_labels rl ON ra.release_id = rl.release_id \
            INNER JOIN release r ON rl.release_id = r.id \
            INNER JOIN label l ON l.name = rl.label \
                 WHERE r.country IN ('US', 'USA & Canada', 'USA', 'United States', 'United States of America') \
                   AND r.genres LIKE '%Country%' \
                   AND l.id IN ({}) \
                   AND SUBSTR(r.released, 1, 4) >= '2018' \
                   AND r.released IS NOT NULL \
                   AND r.released > '' \
              ORDER BY r.released; ").format(major_label_ids_search_string)

#We are going to place less restrictions in the where clause to ensure we're getting all previously signed artists
find_major_releases_prior_2017 = (" SELECT a.id, a.name, r.title, r.genres, \
                       r.status, r.styles, r.released, r.country, \
                       ra.position, l.name AS ""label_name"", l.parent_label, \
                       SUBSTR(r.released, 1, 4) AS ""Year"" \
                  FROM artist a \
            INNER JOIN releases_artists ra ON a.id = ra.artist_id \
            INNER JOIN releases_labels rl ON ra.release_id = rl.release_id \
            INNER JOIN release r ON rl.release_id = r.id \
            INNER JOIN label l ON l.name = rl.label \
                 WHERE r.country IN ('US', 'USA & Canada', 'USA', 'United States', 'United States of America') \
                   AND (SUBSTR(r.released, 1, 4) < '2018' OR  r.released IS NULL OR r.released = '') \
                   AND l.id IN ({}) \
              ORDER BY r.released; ").format(major_label_ids_search_string)

In [2]:
#Get the dataframes we want by executing the queries
try:
#      removed connection information bc it 
#      contained username/password + reran to remove information from print out
    connection = psycopg2.connect() 
    cursor = connection.cursor()
    # Print PostgreSQL Connection properties
    print ( connection.get_dsn_parameters(),"\n")
    
    # Print PostgreSQL version
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record,"\n")
    
    #EXECUTE YOUR QUERY AND SAVE RESULTS IN DATAFRAME
    df_major_country_releases_since_2017 = pd.read_sql_query(find_major_country_releases_since_2017,con=connection)
    df_major_releases_prior_2017 = pd.read_sql_query(find_major_releases_prior_2017,con=connection)
    
except (Exception, psycopg2.Error) as error :
    print("Error while connecting to PostgreSQL", error)
finally:
    #closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

Error while connecting to PostgreSQL missing dsn and no parameters


NameError: name 'connection' is not defined

In [16]:
#Let's get the artist ids for the emerging artists by a set difference operation on artist_id

emerging_artist_ids = set(df_major_country_releases_since_2017.id) - set(df_major_releases_prior_2017.id)

#Now, create a new dataframe for emerging artists only using the set above
df_emerging_country_artists = df_major_country_releases_since_2017.loc[df_major_country_releases_since_2017['id'].isin(emerging_artist_ids)]

print("Length of emerging artist df: ", len(df_emerging_country_artists))
print("Number of emerging artist ids: ", len(df_emerging_country_artists.id.unique()))
df_emerging_country_artists.head(20)

#Find the first major release date for each artist in this dataframe
grouped = df_emerging_country_artists.groupby('name')
print("Date of first major release for emerging artists:")
print(grouped['released'].min())

#Compile a dataframe with distinct names and a 1 for each emerging artist
unique_emerging_signed_artists = list(df_emerging_country_artists['name'].unique())
ones = ["1" for artist in unique_emerging_signed_artists]

#Throw these lists into a dataframe
df_emerging_signed_artists = pd.DataFrame({'artist_name': unique_emerging_signed_artists, 
                                           'y': ones})

#Check to make sure df looks right
df_emerging_signed_artists.head(10)

Length of emerging artist df:  96
Number of emerging artist ids:  56
Date of first major release for emerging artists:
name
Agnes Obel                 2020-02-21
Alyssa Trahan                    2019
Ashley McBryde             2018-03-30
Bobby Krlic                2019-07-05
Brooklyn Rider             2019-10-04
Burna Boy                  2019-07-26
Chelsea Cutler             2019-11-15
City Of The Sun                  2020
Cody Johnson (3)           2019-01-18
County Line Runner         2019-11-29
Craig Campbell (3)               2018
Dailey & Vincent                 2018
Daughter of Swords         2019-06-28
Dee White                        2019
Devon Baldwin                    2019
Dustin Lynch               2020-01-17
Early James                2020-03-13
Ed O'Brien                 2020-03-06
Francesco Turrisi          2019-05-03
Hadley McCall Thackston          2018
Haley & Michaels                 2019
Ingrid Andress                   2019
Jason Aldean               2018-04-13
Je

,artist_name,y
0,Longbranch/Pennywhistle,1
1,Craig Campbell (3),1
2,The Sisterhood Band,1
3,Judee Sill,1
4,Mitchell Tenpenny,1
5,Mountain Man,1
6,Dailey & Vincent,1
7,The Black Lillies,1
8,Tenille Townes,1
9,Hadley McCall Thackston,1


In [4]:
#Now, isolate the artists who have released country music since 2017, but ARE NOT LOCATED in either dataframe

#We will go back to the database to query again. First, we need to grab the artist ids to exclude

artist_ids_with_major_release = set(df_major_country_releases_since_2017.id).union(set(df_major_releases_prior_2017.id))

artist_ids_with_major_release_string = str(list(artist_ids_with_major_release))

artist_ids_with_major_release_string = artist_ids_with_major_release_string[1:-1]

#Compile the query we want to isolate relevant artists
find_country_releases_since_2017_non_major_artists = ("SELECT a.id, a.name, r.title, r.genres,\
                       r.status, r.styles, r.released, r.country,\
                       ra.position, l.name AS ""label_name"", l.parent_label, \
                       SUBSTR(r.released, 1, 4) AS ""Year"" \
                  FROM artist a \
            INNER JOIN releases_artists ra ON a.id = ra.artist_id \
            INNER JOIN releases_labels rl ON ra.release_id = rl.release_id \
            INNER JOIN release r ON rl.release_id = r.id \
            INNER JOIN label l ON l.name = rl.label \
                 WHERE r.country IN ('US', 'USA & Canada', 'USA', 'United States', 'United States of America') \
                   AND r.genres LIKE '%Country%' \
                   AND a.id NOT IN ({}) \
                   AND SUBSTR(r.released, 1, 4) >= '2018' \
                   AND r.released IS NOT NULL \
                   AND r.released > '' \
              ORDER BY r.released; ").format(artist_ids_with_major_release_string)

#Execute the query we have built
try:
#      removed connection information bc it 
#      contained username/password + reran to remove information from print out
    connection = psycopg2.connect()

    cursor = connection.cursor()
    # Print PostgreSQL Connection properties
    print ( connection.get_dsn_parameters(),"\n")
    
    # Print PostgreSQL version
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record,"\n")
    
    #EXECUTE YOUR QUERY AND SAVE RESULTS IN DATAFRAME
    df_country_releases_since_2017_non_major_artists = pd.read_sql_query(find_country_releases_since_2017_non_major_artists,con=connection)
    
except (Exception, psycopg2.Error) as error :
    print("Error while connecting to PostgreSQL", error)
finally:
    #closing database connection.
        if(connection):
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")

NameError: name 'df_major_country_releases_since_2017' is not defined

In [18]:
print("Length of relevant artists with no major releases: ", len(df_country_releases_since_2017_non_major_artists))
print("Number of relevant artists with no major releases ids: ", len(df_country_releases_since_2017_non_major_artists.id.unique()))
#df_country_releases_since_2017_non_major_artists.head(20)

#Compile a dataframe with distinct names of these artists and a 0 to indicate they were never signed
unique_unsigned_artists = list(df_country_releases_since_2017_non_major_artists['name'].unique())
zeros = ["0" for artist in unique_unsigned_artists]

#Throw these lists into a dataframe
df_unsigned_artists = pd.DataFrame({'artist_name': unique_unsigned_artists, 
                                           'y': zeros})

#Check to make sure df looks right
df_unsigned_artists.head(10)

Length of relevant artists with no major releases:  7403
Number of relevant artists with no major releases ids:  4231


,artist_name,y
0,Brothers Egg,0
1,Adam Wright,0
2,Hard Luck Country Club,0
3,Loren Mazzacane Connors,0
4,Root Marm Chicken Farm Jug Band,0
5,John Craigie,0
6,Midnight River Choir,0
7,Andras Jones,0
8,Dylan Blanchard,0
9,The Fossil Lickers,0


In [22]:
#Concatenate the two dataframes that have my artists and y labels
df_artists_with_y_labels = pd.concat([df_emerging_signed_artists, df_unsigned_artists])

#Print dataframe to make sure it looks right
df_artists_with_y_labels.head(10)

#Export it as csv to hand off as input for next project task
df_artists_with_y_labels.to_csv('artists_with_y_labels.csv', index=False, header=True)

IGNORE FROM THIS POINT DOWN. KEEPING FOR REFERENCE BUT NOT CURRENTLY BEING USED.

In [ ]:
#Print the columns for each Discogs table for reference
print(df_artist.columns)
print(df_label.columns)
print(df_release.columns)
print(df_releases_artists.columns)
print(df_releases_labels.columns)

In [7]:
#Create a function to return if a release is apart of major label (1) or not (0)
def is_major(label_name_col, parent_label_col):
    y_rec_label = []
    for label, parent in zip(list(label_name_col), list(parent_label_col)):
        if label in major_set or parent in major_set:
            y_rec_label.append(1)
        else:
            y_rec_label.append(0)
    return y_rec_label

In [8]:
#Pass the label and parent columns from my example df to the function.
y_labels = is_major(df_artist_release_information.label_name, df_artist_release_information.parent_label)

#Add this list into the dataframe as y_rec_label
df_artist_release_information['y_rec_label'] = y_labels
#Show the dataframe again


,id,name,title,genres,status,styles,released,country,position,label_name,parent_label,Year,y_rec_label
0,5940000,Luke Combs,This One's For You,"{""Folk, World, & Country""}",Accepted,{Country},2017,US,1,Columbia Nashville,None,2017,1
1,5940000,Luke Combs,This One's For You,"{""Folk, World, & Country""}",Accepted,{Country},2017-06-02,US,1,Sony Music Nashville,Sony Music Entertainment,2017,1
2,5940000,Luke Combs,This One's For You,"{""Folk, World, & Country""}",Accepted,{Country},2017-06-09,US,1,Columbia Nashville,None,2017,1
3,130060,Shania Twain,Now,"{Pop,""Folk, World, & Country""}",Accepted,None,2017-09-29,US,1,Mercury Nashville,Universal Music Group,2017,1
4,130060,Shania Twain,Now,"{Pop,""Folk, World, & Country""}",Accepted,None,2017-09-29,US,1,Mercury Nashville,Universal Music Group,2017,1
5,130060,Shania Twain,Now,"{Pop,""Folk, World, & Country""}",Accepted,"{Country,Ballad,Vocal}",2017-09-29,US,1,Mercury Nashville,Universal Music Group,2017,1
6,6884886,Tenille Townes,Living Room Worktapes,"{""Folk, World, & Country""}",Accepted,{Country},2018,US,1,Columbia Nashville,None,2018,1
7,5940000,Luke Combs,This One's For You Too,"{Rock,""Folk, World, & Country""}",Accepted,"{Country,""Country Rock""}",2018-06-01,US,1,Columbia Nashville,None,2018,1
8,5940000,Luke Combs,This One's For You Too,"{Rock,""Folk, World, & Country""}",Accepted,"{Country,""Country Rock""}",2018-11-16,US,1,Columbia Nashville,None,2018,1
9,5940000,Luke Combs,Beer Never Broke My Heart,"{""Folk, World, & Country""}",Accepted,{Country},2019-05-08,US,1,Columbia Nashville,None,2019,1
